# Gender in the Library of Congress Subject Headings

### Imports

In [1]:
import pickle as pk
import random
import matplotlib.pyplot as plt 
import numpy as np
import re
import csv
import collections

import spacy 
nlp = spacy.load("en_core_web_sm") 

### Functions

In [46]:
def getNew(term):
    return int(term['yearNew'][:4])

def percent(num, denom):
    if denom == 0:
        return 0
    return num/denom*100

def findTerms(pattern, terms):
    matchedTerms = {}
    for idx, term in terms.items():
        if re.search(pattern, term['heading'].lower()) is not None:
            matchedTerms[idx] = term
    return matchedTerms    

def clean(word, keepBrack=False):
    if keepBrack:
        return word.lower().replace("'", " '").replace(',', '').replace('_', '').replace('-', ' ').strip()
    else:
        return word.lower().replace('(', '').replace(')', '').replace("'", " '").replace(',', '').replace('_', '').replace('-', ' ').strip()

def findPairs(mTerms, wTerms, pairs):
    pairedTerms = []
    indicesW, indicesM = [], []
    mHeads = [t['heading'] for t in mTerms.values()]
    wHeads = [t['heading'] for t in wTerms.values()]
    for i in range(len(pairs)):
        wordM, wordW = pairs[i]
        replaced = {}
        for i, head in enumerate(wHeads):
            wHead = head.lower()
            if wordW in wHead:
                replaced[wHead.replace(wordW, '')] = i
        for i, head in enumerate(mHeads):
            mHead = head.lower()
            if wordM in mHead:
                genTerm = mHead.replace(wordM, '')
                if genTerm in replaced.keys() and i not in indicesM and replaced[genTerm] not in indicesW:
                    pairedTerms.append((list(mTerms.items())[i],list(wTerms.items())[replaced[genTerm]]))
                    indicesM.append(i)
                    indicesW.append(replaced[genTerm])
    return pairedTerms

### Load Data

In [222]:
with open('Data/LCSH/lcsh.pickle', 'rb') as f:
    lcsh = pk.load(f)

with open('Data/LCSH/genLCSH-current.pickle', 'rb') as f:
    wTerms, mTerms = pk.load(f)

#### [WOMEN/MEN] in [BLANK]

In [4]:
menIn = findTerms(r'^men in ', mTerms)
womenIn = findTerms(r'^women in ', wTerms)

menInSub = {idx:term for idx, term in menIn.items() if term['bt'] is not None}
womenInSub = {idx:term for idx, term in womenIn.items() if term['bt'] is not None}

matched = findPairs(menInSub, womenInSub, [('men', 'women')])
matchIdxW = [w[0] for _, w in matched]
matchIdxM = [m[0] for m,_ in matched]

print(f'There are {len(menIn)} terms of the form "men in __" and {len(menInSub)} ({percent(len(menInSub), len(menIn)):.2f}%) have a broader term.')
print(f'There are {len(womenIn)} terms of the form "women in __" and {len(womenInSub)} ({percent(len(womenInSub), len(womenIn)):.2f}%) have a broader term.')

menIn_mark = [term for idx, term in menInSub.items() if idx not in matchIdxM]
womenIn_mark = [term for idx, term in womenInSub.items() if idx not in matchIdxW]

print(f'There are {len(menIn_mark)} terms were men are the marked case and {len(womenIn_mark)} where women are. {len(matched)} are paired.')

There are 11 terms of the form "men in __" and 8 (72.73%) have a broader term.
There are 131 terms of the form "women in __" and 121 (92.37%) have a broader term.
There are 0 terms were men are the marked case and 113 where women are. 8 are paired.


#### [Adj] [WOMEN/MEN] in [BLANK]

In [136]:
adjMenIn = findTerms(r' men in ', mTerms)
adjWomenIn = findTerms(r' women in ', wTerms)

adjMenInSub = {idx:term for idx, term in adjMenIn.items() if term['bt'] is not None}
adjWomenInSub = {idx:term for idx, term in adjWomenIn.items() if term['bt'] is not None}

matched = findPairs(adjMenInSub, adjWomenInSub, [('men', 'women')])
matchIdxW = [w[0] for _, w in matched]
matchIdxM = [m[0] for m,_ in matched]

print(f'There are {len(adjMenIn)} terms of the form "[Adj] men in __" and {len(adjMenInSub)} ({percent(len(adjMenInSub), len(adjMenIn)):.2f}%) have a broader term.')
print(f'There are {len(adjWomenIn)} terms of the form "[Adj] women in __" and {len(adjWomenInSub)} ({percent(len(adjWomenInSub), len(adjWomenIn)):.2f}%) have a broader term.')

adjMenIn_mark = [term for idx, term in adjMenInSub.items() if idx not in matchIdxM]
adjWomenIn_mark = [term for idx, term in adjWomenInSub.items() if idx not in matchIdxW]

print(f'There are {len(adjMenIn_mark)} terms were men are the marked case and {len(adjWomenIn_mark)} where women are. {len(matched)} are paired.')

There are 25 terms of the form "[Adj] men in __" and 11 (44.00%) have a broader term.
There are 85 terms of the form "[Adj] women in __" and 41 (48.24%) have a broader term.
There are 6 terms were men are the marked case and 36 where women are. 5 are paired.


[Women/Men] [Noun Phrase]

In [232]:
def checkIdx(idx, lcsh): 
    try: 
        lcsh[idx]
        return True
    except KeyError: 
        return False

In [230]:
def findTermsPOS(pattern, indexWord, terms):
    matchedTerms = {}
    for idx, term in terms.items():
        if re.search(pattern, term['heading'].lower()) is not None:
            words = clean(term['heading'], keepBrack=True).split()
            i = words.index(indexWord) + 1
            pos = nlp(words[i])[0].tag_
            if ('(' not in words[i] 
                    and pos != 'DT' 
                    and pos != 'IN' 
                    and pos != 'CC' 
                    and 'in' not in words
                    and 'on' not in words):
                matchedTerms[idx] = term
    return matchedTerms
            
def getParents(term, lcsh):
    if term['bt'] is not None:
        parents = [lcsh[idx] for idx in term['bt'] if idx.strip()]
    else:
        parents = []
    return parents

def getSiblings(term, parent, lcsh):
    heads = [lcsh[idx]['heading'] for idx in parent['nt'] if idx.strip() and checkIdx(idx, lcsh)]
    if term['heading'] not in heads:
        raise ValueError('Parent LCSH is not the parent of the selected LCSH')
    elif len(heads) == 1:
        return []
    else:
        return [lcsh[idx] for idx in parent['nt'] if idx.strip() and checkIdx(idx, lcsh) and lcsh[idx]['heading'] != term['heading']]
    
def getSpecialTerm(siblings, keyWords, disWords):
    specialSibs = []
    for sib in siblings:
        flag = False
        if sib['synonyms'] is not None and sib['type'] == 'Topic':
            words = clean(' '.join([var for var in sib['synonyms'] if '--' not in var])).split()
            for word in keyWords:
                if word in words:
                    flag = True
                    for word in disWords:
                        if word in words:
                            flag = False
                    break
            if flag:
                specialSibs.append(sib)
    return specialSibs


            

In [226]:
menBlanks = findTermsPOS(r'^men ', 'men', mTerms)
womenBlanks = findTermsPOS(r'^women ', 'women', wTerms)

menBlanksSub = {idx:term for idx, term in menBlanks.items() if term['bt'] is not None}
womenBlanksSub = {idx:term for idx, term in womenBlanks.items() if term['bt'] is not None}

matchedW = findPairs(menBlanksSub, womenBlanksSub, [('men', 'women')])
matchIdxW_ = [w[0] for _, w in matchedW]
matchIdxM = [m[0] for m,_ in matchedW]

print(f'There are {len(menBlanks)} terms of the form "men __" and {len(menBlanksSub)} ({percent(len(menBlanksSub), len(menBlanks)):.2f}%) have a broader term.')
print(f'There are {len(womenBlanks)} terms of the form "women __" and {len(womenBlanksSub)} ({percent(len(womenBlanksSub), len(womenBlanks)):.2f}%) have a broader term.')

menBlanks_mark = [term for idx, term in menBlanksSub.items() if idx not in matchIdxM]
womenBlanks_mark = [term for idx, term in womenBlanksSub.items() if idx not in matchIdxW_]

print(f'There are {len(menBlanks_mark)} terms were men are the marked case and {len(womenBlanks_mark)} where women are. {len(matchedW)} are paired.')

There are 0 terms of the form "men __" and 0 (0.00%) have a broader term.
There are 925 terms of the form "women __" and 697 (75.35%) have a broader term.
There are 0 terms were men are the marked case and 697 where women are. 0 are paired.


In [140]:
adjMenBlanks = findTermsPOS(r' men ', 'men', mTerms)
adjWomenBlanks = findTermsPOS(r' women ', 'women', wTerms)

adjMenBlanksSub = {idx:term for idx, term in adjMenBlanks.items() if term['bt'] is not None}
adjWomenBlanksSub = {idx:term for idx, term in adjWomenBlanks.items() if term['bt'] is not None}

matched = findPairs(adjMenBlanksSub, adjWomenBlanksSub, [('men', 'women')])
matchIdxW = [w[0] for _, w in matched]
matchIdxM = [m[0] for m,_ in matched]

print(f'There are {len(adjMenBlanks)} terms of the form "[adj] men __" and {len(adjMenBlanksSub)} ({percent(len(adjMenBlanksSub), len(adjMenBlanks)):.2f}%) have a broader term.')
print(f'There are {len(adjWomenBlanks)} terms of the form "[adj] women __" and {len(adjWomenBlanksSub)} ({percent(len(adjWomenBlanksSub), len(adjWomenBlanks)):.2f}%) have a broader term.')

adjMenBlanks_mark = [term for idx, term in adjMenBlanksSub.items() if idx not in matchIdxM]
adjWomenBlanks_mark = [term for idx, term in adjWomenBlanksSub.items() if idx not in matchIdxW]

print(f'There are {len(adjMenBlanks_mark)} terms were men are the marked case and {len(adjWomenBlanks_mark)} where women are. {len(matched)} are paired.')

There are 0 terms of the form "[adj] men __" and 0 (0.00%) have a broader term.
There are 228 terms of the form "[adj] women __" and 228 (100.00%) have a broader term.
There are 0 terms were men are the marked case and 228 where women are. 0 are paired.


In [153]:
maleBlanks = findTermsPOS(r'^male ', 'male', mTerms)
femaleBlanks = findTermsPOS(r'^female ', 'female', wTerms)

maleBlanksSub = {idx:term for idx, term in maleBlanks.items() if term['bt'] is not None}
femaleBlanksSub = {idx:term for idx, term in femaleBlanks.items() if term['bt'] is not None}

matchedF = findPairs(maleBlanksSub, femaleBlanksSub, [('male', 'female')])

matchIdxW = [w[0] for _, w in matchedF]
matchIdxM_ = [m[0] for m,_ in matchedF]

print(f'There are {len(maleBlanks)} terms of the form "male __" and {len(maleBlanksSub)} ({percent(len(maleBlanksSub), len(maleBlanks)):.2f}%) have a broader term.')
print(f'There are {len(femaleBlanks)} terms of the form "female __" and {len(femaleBlanksSub)} ({percent(len(femaleBlanksSub), len(femaleBlanks)):.2f}%) have a broader term.')

maleBlanks_mark = [term for idx, term in maleBlanksSub.items() if idx not in matchIdxM_]
femaleBlanks_mark = [term for idx, term in femaleBlanksSub.items() if idx not in matchIdxW]

print(f'There are {len(maleBlanks_mark)} terms were men are the marked case and {len(femaleBlanks_mark)} where women are. {len(matchedF)} are paired.')

There are 64 terms of the form "male __" and 61 (95.31%) have a broader term.
There are 23 terms of the form "female __" and 23 (100.00%) have a broader term.
There are 56 terms were men are the marked case and 18 where women are. 5 are paired.


There are pairs of terms where 'male' is matched with 'women' so I'll look at terms for 'men' and 'male' vs 'women' and 'female'

In [192]:
matched = findPairs(maleBlanksSub, womenBlanksSub, [('male', 'women')])
matchIdxW = [w[0] for _, w in matched]
matchIdxM = [m[0] for m,_ in matched]
matched += matchedF + matchedW

mmBlanks_mark = [term for idx, term in maleBlanksSub.items() if idx not in matchIdxM and idx not in matchIdxM_] + menBlanks_mark
wfBlanks_mark = [term for idx, term in womenBlanksSub.items() if idx not in matchIdxW and idx not in matchIdxW_] + femaleBlanks_mark

print(f'There are {len(mmBlanks_mark)} terms were men are the marked case (as male or men) and {len(wfBlanks_mark)} where women are (as women or female). {len(matched)} are paired.')

There are 29 terms were men are the marked case (as male or men) and 688 where women are (as women or female). 32 are paired.


In [201]:
with open('Data/markedMen.txt', 'w') as f:   
    for term in mmBlanks_mark:
        parents = getParents(term, lcsh)
        f.write(f'{term['heading']} ({getNew(term)}):\n')
        for i, parent in enumerate(parents):
            f.write(f'\tParent {i+1}: {parent['heading']} ({getNew(parent)})\n')
            parentWords = clean(parent['heading']).split()
            if 'male' not in parentWords and 'men' not in parentWords:
                siblings = getSiblings(term, parent, lcsh)
                if len(siblings) > 0:
                    siblingStr = f'\t\tSiblings:\n'
                    for sibling in siblings:
                        siblingStr = siblingStr + f'\t\t\t{sibling['heading']} ({getNew(sibling)});\n'
                    f.write(siblingStr)
        f.write('\n')

In [233]:
with open('Data/markedWomen.txt', 'w') as f:   
    for term in wfBlanks_mark:
        parents = getParents(term, lcsh)
        f.write(f'{term['heading']} ({getNew(term)}):\n')
        for i, parent in enumerate(parents):
            f.write(f'\tParent {i+1}: {parent['heading']} ({getNew(parent)})\n')
            parentWords = clean(parent['heading']).split()
            if 'female' not in parentWords and 'women' not in parentWords:
                siblings = getSiblings(term, parent, lcsh)
                if len(siblings) > 0:
                    siblingStr = f'\t\tSiblings:\n'
                    for sibling in siblings:
                        siblingStr = siblingStr + f'\t\t\t{sibling['heading']} ({getNew(sibling)});\n'
                    f.write(siblingStr)
        f.write('\n')

I want to update this to improve my method for finding words that are gendered variants that are obscured because they don't explicitly identify men or women (i.e. male actors and actresses)

In [235]:
for term in wfBlanks_mark:
    parents = getParents(term, lcsh)
    for i, parent in enumerate(parents):
        parentWords = clean(parent['heading']).split()
        if 'female' not in parentWords and 'women' not in parentWords:
            siblings = getSiblings(term, parent, lcsh)
            if len(siblings) > 0:
                specSibs = getSpecialTerm(siblings, ['men', 'male'], ['women', 'female'])
                if len(specSibs) > 0:
                    print(term['heading'], [s['heading'] for s in specSibs])

Women transport workers ['Rickshaw men']
Women fishers ['Whalers (Persons)']
Women healers ['Shamans']
Women television personalities ['Television news anchors']
Women mediums ['Shamans']
Women television journalists ['Television news anchors']
Female genital mutilation ['Circumcision']
Female genital mutilation ['Circumcision']


In [234]:
for term in mmBlanks_mark:
    parents = getParents(term, lcsh)
    for i, parent in enumerate(parents):
        parentWords = clean(parent['heading']).split()
        if 'male' not in parentWords and 'men' not in parentWords:
            siblings = getSiblings(term, parent, lcsh)
            if len(siblings) > 0:
                specSibs = getSpecialTerm(siblings, ['women', 'female'], ['men', 'male'])
                if len(specSibs) > 0:
                    print(term['heading'], [s['heading'] for s in specSibs])

Male homosexuality ['Lesbianism']
Male actors ['Actresses']
Male prostitutes ['Camp followers']
Male sexual abuse victims ['Rape victims']
